# Introduction to TutorialD via Jupyter Notebook

Thanks for trying Project:M36 via this Jupyter notebook. TutorialD is a language for interacting with relational algebra databases, similar to SQL but without the historical cruft. This notebook is based on the [TutorialD Tutorial](https://github.com/agentm/project-m36/blob/master/docs/tutd_tutorial.markdown). 

[Project:M36](https://github.com/agentm/project-m36) is a feature-rich database management system (DBMS) which casts off the baggage of legacy DBMSes by adhering strictly to the mathematics of the relational algebra.

Please refer to the [Project:M36 documentation](https://github.com/agentm/project-m36#documentation) for more detailed examples. Let's get started!

First, let's import the Chris Date example relation variables. The relation variables provided in the example are:

* `s` suppliers
* `sp` supplier-parts
* `p` parts

In [1]:
:importexample date

ok

If you see an error in the previous cell regarding `project-m36-websocket-server` not being in your `PATH`, add the binary to your `PATH` and restart the jupyter server.

Let's take a look at the relation variables.

In [2]:
:showexpr s

┌──────────┬────────┬───────────┬───────────────┐
│city::Text│s#::Text│sname::Text│status::Integer│
├──────────┼────────┼───────────┼───────────────┤
│"Paris"   │"S2"    │"Jones"    │10             │
│"London"  │"S1"    │"Smith"    │20             │
│"London"  │"S4"    │"Clark"    │20             │
│"Athens"  │"S5"    │"Adams"    │30             │
│"Paris"   │"S3"    │"Blake"    │30             │
└──────────┴────────┴───────────┴───────────────┘

In [3]:
:showexpr sp

s#::Text,p#::Text,qty::Integer
"""S1""","""P2""",200
"""S1""","""P1""",300
"""S3""","""P2""",200
"""S2""","""P2""",400
"""S4""","""P5""",400
"""S1""","""P5""",100
"""S1""","""P6""",100
"""S2""","""P1""",300
"""S1""","""P4""",200
"""S4""","""P4""",300


In [4]:

:showexpr p

┌──────────┬───────────┬────────┬───────────┬───────────────┐
│city::Text│color::Text│p#::Text│pname::Text│weight::Integer│
├──────────┼───────────┼────────┼───────────┼───────────────┤
│"Paris"   │"Blue"     │"P5"    │"Cam"      │12             │
│"London"  │"Red"      │"P1"    │"Nut"      │12             │
│"London"  │"Red"      │"P6"    │"Cog"      │19             │
│"London"  │"Red"      │"P4"    │"Screw"    │14             │
│"Paris"   │"Green"    │"P2"    │"Bolt"     │17             │
│"Oslo"    │"Blue"     │"P3"    │"Screw"    │17             │
└──────────┴───────────┴────────┴───────────┴───────────────┘

## Relational Operators

### Restriction

We can see that there is a many-to-many relationship between `s` (suppliers) and `p` (parts) via the `sp` relation variable.

Let's look at the suppliers who are in London. This is called "restriction".

Note that the relation attributes have appropriate types which restrict the values which they can contain.

In [5]:
:showexpr s where city="London"

┌──────────┬────────┬───────────┬───────────────┐
│city::Text│s#::Text│sname::Text│status::Integer│
├──────────┼────────┼───────────┼───────────────┤
│"London"  │"S1"    │"Smith"    │20             │
│"London"  │"S4"    │"Clark"    │20             │
└──────────┴────────┴───────────┴───────────────┘

### Join And Projection

Let's examine which parts the London suppliers offer. In the following expression, we make use of the relational join operator (equivalent to `NATURAL JOIN` in SQL) and projection using the trailing curly brackets to only return the relation attributes which interest us.

In [6]:
:showexpr ((s where city="London") join sp join p){p#,pname,qty}

┌────────┬───────────┬────────────┐
│p#::Text│pname::Text│qty::Integer│
├────────┼───────────┼────────────┤
│"P4"    │"Screw"    │200         │
│"P4"    │"Screw"    │300         │
│"P1"    │"Nut"      │300         │
│"P6"    │"Cog"      │100         │
└────────┴───────────┴────────────┘

### Extend and Atom Functions

We can also execute server-side functions on values and add them to our result relation.

In [7]:
:showexpr (s:{statusplus10:=add(@status,10)}){s#,status,statusplus10}

┌────────┬───────────────┬─────────────────────┐
│s#::Text│status::Integer│statusplus10::Integer│
├────────┼───────────────┼─────────────────────┤
│"S2"    │10             │20                   │
│"S3"    │30             │40                   │
│"S5"    │30             │40                   │
│"S4"    │20             │30                   │
│"S1"    │20             │30                   │
└────────┴───────────────┴─────────────────────┘

### Group and Ungroup

Unlike most databases, Project:M36 supports relation-valued attributes. This means that database values (atoms) can also be relations. Relation-valued attributes can be represented in the notebook as nested tables of values. Note that the nested relation's type is `relation {qty::Integer,pname::Text,s#::Text}`. 

Relation-valued attributes reduce tuple duplication by clearly associating certain values with their related multiple tuples. The `group` operator can also be used to create the SQL equivalent of `OUTER JOIN` but without the need for representing `NULL` in any form.

Let's list the available counts for each supplier for each part.

In [8]:
:showexpr (p join sp){qty,pname,s#,p#} group ({s#,pname,qty} as availability)

┌──────────────────────────────────────────────────────────┬────────┐
│availability::relation {qty::Integer,pname::Text,s#::Text}│p#::Text│
├──────────────────────────────────────────────────────────┼────────┤
│┌───────────┬────────────┬────────┐                       │"P5"    │
││pname::Text│qty::Integer│s#::Text│                       │        │
│├───────────┼────────────┼────────┤                       │        │
││"Cam"      │100         │"S1"    │                       │        │
││"Cam"      │400         │"S4"    │                       │        │
│└───────────┴────────────┴────────┘                       │        │
│┌───────────┬────────────┬────────┐                       │"P3"    │
││pname::Text│qty::Integer│s#::Text│                       │        │
│├───────────┼────────────┼────────┤                       │        │
││"Screw"    │400         │"S1"    │                       │        │
│└───────────┴────────────┴────────┘                       │        │
│┌───────────┬────────────┬────────┐                       │"P4"    │
││pname::Text│qty::Integer│s#::Text│                       │        │
│├───────────┼────────────┼────────┤                       │        │
││"Screw"    │300         │"S4"    │                       │        │
││"Screw"    │200         │"S1"    │                       │        │
│└───────────┴────────────┴────────┘                       │        │
│┌───────────┬────────────┬────────┐                       │"P2"    │
││pname::Text│qty::Integer│s#::Text│                       │        │
│├───────────┼────────────┼────────┤                       │        │
││"Bolt"     │200         │"S3"    │                       │        │
││"Bolt"     │400         │"S2"    │                       │        │
││"Bolt"     │200         │"S1"    │                       │        │
││"Bolt"     │200         │"S4"    │                       │        │
│└───────────┴────────────┴────────┘                       │        │
│┌───────────┬────────────┬────────┐                       │"P1"    │
││pname::Text│qty::Integer│s#::Text│                       │        │
│├───────────┼────────────┼────────┤                       │        │
││"Nut"      │300         │"S1"    │                       │        │
││"Nut"      │300         │"S2"    │                       │        │
│└───────────┴────────────┴────────┘                       │        │
│┌───────────┬────────────┬────────┐                       │"P6"    │
││pname::Text│qty::Integer│s#::Text│                       │        │
│├───────────┼────────────┼────────┤                       │        │
││"Cog"      │100         │"S1"    │                       │        │
│└───────────┴────────────┴────────┘                       │        │
└──────────────────────────────────────────────────────────┴────────┘

We can flatten the relation-valued attributes, thereby generating more top-level tuples.

In [9]:
:showexpr ((p join sp){qty,pname,s#,p#} group ({s#,pname,qty} as availability)) ungroup availability

qty::Integer,pname::Text,s#::Text,p#::Text
400,"""Screw""","""S1""","""P3"""
200,"""Bolt""","""S4""","""P2"""
100,"""Cog""","""S1""","""P6"""
100,"""Cam""","""S1""","""P5"""
300,"""Nut""","""S2""","""P1"""
400,"""Cam""","""S4""","""P5"""
300,"""Screw""","""S4""","""P4"""
200,"""Bolt""","""S1""","""P2"""
200,"""Screw""","""S1""","""P4"""
200,"""Bolt""","""S3""","""P2"""


## Database State Operators

Thus far, we have only been inspecting the database state. Next, let's cover some database update operators. These operators are all similar to SQL operators, so we don't need a lot of exposition.

### Insert

In [10]:
insert s relation{tuple{s# "S6", sname "Bob", status 50, city "Boston"}}

ok

In [11]:
:showexpr s

┌──────────┬────────┬───────────┬───────────────┐
│city::Text│s#::Text│sname::Text│status::Integer│
├──────────┼────────┼───────────┼───────────────┤
│"Paris"   │"S2"    │"Jones"    │10             │
│"London"  │"S1"    │"Smith"    │20             │
│"London"  │"S4"    │"Clark"    │20             │
│"Athens"  │"S5"    │"Adams"    │30             │
│"Paris"   │"S3"    │"Blake"    │30             │
│"Boston"  │"S6"    │"Bob"      │50             │
└──────────┴────────┴───────────┴───────────────┘

### Delete

In [12]:
delete s where sname="Bob"

ok

In [13]:
:showexpr s

┌──────────┬────────┬───────────┬───────────────┐
│city::Text│s#::Text│sname::Text│status::Integer│
├──────────┼────────┼───────────┼───────────────┤
│"Paris"   │"S2"    │"Jones"    │10             │
│"London"  │"S1"    │"Smith"    │20             │
│"London"  │"S4"    │"Clark"    │20             │
│"Athens"  │"S5"    │"Adams"    │30             │
│"Paris"   │"S3"    │"Blake"    │30             │
└──────────┴────────┴───────────┴───────────────┘

### Update

In [14]:
update s where city="London" (status:=90)

ok

In [15]:
:showexpr s

┌──────────┬────────┬───────────┬───────────────┐
│city::Text│s#::Text│sname::Text│status::Integer│
├──────────┼────────┼───────────┼───────────────┤
│"Paris"   │"S2"    │"Jones"    │10             │
│"London"  │"S4"    │"Clark"    │90             │
│"Athens"  │"S5"    │"Adams"    │30             │
│"Paris"   │"S3"    │"Blake"    │30             │
│"London"  │"S1"    │"Smith"    │90             │
└──────────┴────────┴───────────┴───────────────┘

### Assign

In [16]:
london_suppliers := s where city="London"

ok

In [17]:
:showexpr london_suppliers

┌──────────┬────────┬───────────┬───────────────┐
│city::Text│s#::Text│sname::Text│status::Integer│
├──────────┼────────┼───────────┼───────────────┤
│"London"  │"S4"    │"Clark"    │90             │
│"London"  │"S1"    │"Smith"    │90             │
└──────────┴────────┴───────────┴───────────────┘

## Constraints

Project:M36 supports too many features to cover in one notebook, but constraints are certainly essential.

Let's examine the current constraints.

In [18]:
:constraints

┌──────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────┐
│name::Text│sub::Text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          │super::Text                                                      │
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼─────────────────────────────────────────────────────────────────┤
│"sp_pkey" │"NotEquals (Project (AttributeNames (fromList [\"b\"])) (Extend (AttributeExtendTupleExpr \"b\" (FunctionAtomExpr \"count\" [AttributeAtomExpr \"a\"] ())) (Extend (AttributeExtendTupleExpr \"a\" (RelationAtomExpr (RelationVariable \"sp\" ()))) (ExistingRelation (Relation [] (RelationTupleSet {asList = [RelationTuple [] []]})))))) (Project (AttributeNames (fromList [\"b\"])) (Extend (AttributeExtendTupleExpr \"b\" (FunctionAtomExpr \"count\" [AttributeAtomExpr \"a\"] ())) (Extend (AttributeExtendTupleExpr \"a\" (RelationAtomExpr (Project (AttributeNames (fromList [\"p#\",\"s#\"])) (RelationVariable \"sp\" ())))) (ExistingRelation (Relation [] (RelationTupleSet {asList = [RelationTuple [] []]}))))))"│"ExistingRelation (Relation [] (RelationTupleSet {asList = []}))"│
│"s_pkey"  │"NotEquals (Project (AttributeNames (fromList [\"b\"])) (Extend (AttributeExtendTupleExpr \"b\" (FunctionAtomExpr \"count\" [AttributeAtomExpr \"a\"] ())) (Extend (AttributeExtendTupleExpr \"a\" (RelationAtomExpr (RelationVariable \"s\" ()))) (ExistingRelation (Relation [] (RelationTupleSet {asList = [RelationTuple [] []]})))))) (Project (AttributeNames (fromList [\"b\"])) (Extend (AttributeExtendTupleExpr \"b\" (FunctionAtomExpr \"count\" [AttributeAtomExpr \"a\"] ())) (Extend (AttributeExtendTupleExpr \"a\" (RelationAtomExpr (Project (AttributeNames (fromList [\"s#\"])) (RelationVariable \"s\" ())))) (ExistingRelation (Relation [] (RelationTupleSet {asList = [RelationTuple [] []]}))))))"         │"ExistingRelation (Relation [] (RelationTupleSet {asList = []}))"│
│"p_pkey"  │"NotEquals (Project (AttributeNames (fromList [\"b\"]

Constraints are represented as "inclusion dependencies". An inclusion dependency is a generalization of all possible constraints on the database and can represent keys, uniqueness constraints, foreign key constraints, as well as span multiple relation variables.

An inclusion dependency requires that the result of relational expression `sub` is a subset of relational expression `super`.

Project:M36 includes some utilities to generate inclusion dependencies.

In [19]:
sandwich := relation{tuple{name "Cheese", price 4},tuple{name "Tuna", price 5}}

ok

In [20]:
key sandwich_name{name} sandwich

ok

In [21]:
insert sandwich relation{tuple{name "Tuna", price 6}}

TutorialDError: 

## Jupyter Kernel Features

By default, the TutorialD kernel will automatically start a `project-m36-websocket-server` when the first TutorialD command is executed. For this to work, `project-m36-websocket-server` must be in the jupyter server's `PATH` environment variable.

### Magic

Like the python kernel, the TutorialD kernel includes some commands to interact directly with the kernel. These commands are not passed to the TutorialD interpreter.

* `%help` brings up some help which includes links to additional documentation
* `%connect ws://<host>:<port> <dbname>` connects to the `project-m36-websocket-server` already running. For example:
  ```
  %connect ws://localhost:64000 employees
  ```
  connects to the websocket server running on the localhost on port 64000 and selects the database named "employees". Note that the database name is not part of the websocket URL.
  
## Conclusion

[Project:M36](https://github.com/agentm/project-m36) is a fully-featured relational algebra engine suitable for use a database. It sets out to prove that sticking to the mathematical underpinnings of the relational algebra is straightforward and results in improved correctness and performance.

If you would like to learn more, please refer to the [documentation](https://github.com/agentm/project-m36#documentation) and join our welcoming [community](https://github.com/agentm/project-m36#community) to ask questions.